# Useful scripts for later use 
###(not part of current development)

### Checking symmetry of a MATLAB solution

In [1]:
from flowFieldWavy import *
import numpy as np
vF,pF = mapData2ff(eps=0.02,g=1.0,Re=100., theta=0)[:2]
a = vF.flowDict['alpha']
for x in np.arange(0.,np.pi/a,np.pi/a/21.):
    print(np.linalg.norm(vF.ifft(xLoc=x)[0] - vF.ifft(xLoc=x+np.pi/a)[0,::-1]) )
#print(vF.ifft(xLoc=0))

1.60125554082e-13
1.60112644336e-13
1.60084459486e-13
1.60042401126e-13
1.59982019586e-13
1.59904635808e-13
1.59901654358e-13
1.59916770834e-13
1.59962648591e-13
1.59790109011e-13
1.59831221832e-13
1.59708190707e-13
1.5977536299e-13
1.59932902939e-13
1.5973343672e-13
1.59897972038e-13
1.59886576258e-13
1.60055238894e-13
1.59960423239e-13
1.60105232668e-13
1.60111730526e-13


## Solving riblet flow without coordinate transformation

The velocity, $u$, is expanded as

$a_0 + b_0 y + \sum_n [a_n e^{-ny} + b_n e^{ny}] e^{in\beta z}$

For each mode, there are two unknowns, a and b. The boundary conditions are that the velocity is zero at either wall. So, if I set the above expression to zero for some 'm' locations, and use 'm' modes (including zero), I can simply invert the coefficient matrix to obtain all the 'a's and 'b's. 

In [ ]:
import numpy as np

b = 10. # beta
eps = 0.02
n = 21 # Number of positive modes
m = 2*n+1 # Total number of modes including zero



zArr = np.arange(0., 2.*np.pi/b-1.0e-9, 2.*np.pi/b/m)
yShape = 2.*eps*np.cos(b*zArr)
L = np.zeros((2*m, 2*m), dtype=np.complex)

yTop = (1.+yShape).reshape((yShape.size,1))
yBottom = -1.+yShape.reshape((yShape.size,1))
zArr = zArr.reshape((zArr.size,1))

expTopA = -yTop + 1.j*b*zArr
expTopB = yTop + 1.j*b*zArr
expBottomA = -yBottom + 1.j*b*zArr
expBottomB = yBottom + 1.j*b*zArr

nArr = np.arange(-n,n+1).reshape((1,m))

L[:m, :-1:2] = np.exp(nArr*expTopA)       # Factors multiplying a_n, for yTop locations
L[:m, 1::2] = np.exp(nArr*expTopB)       # Factors multiplying b_n, for yTop locations
L[m:, :-1:2] = np.exp(nArr*expBottomA)    # yBottom locations
L[m:, 1::2] = np.exp(nArr*expBottomB)

# For a_0, the factor is 1 = exp(0.) = exp(0.*y + 0.*1.j*b*z), so above assignment works
# For b_0, the factor is y, but the above assignment only assigns a factor of 1
# Correcting factors for b_0
L[:m, 2*n+1:2*n+2] += yTop - 1.
L[m:, 2*n+1:2*n+2] += yBottom - 1.

# To find the factors, a_n, b_n, the null-space of L needs to be found 
#           (since the boundary condition at yTop, yBottom is that velocity is zero)
# Before that, let's see what the rank is
print('Size of L[0]:', L.shape[0])
print('Rank of L is', np.linalg.matrix_rank(L))
np.linalg.null?

## Script for flow-vis: theta = 90 (old version)
### Latest implementation is to use flowFieldArray (see next section)

In [ ]:
import numpy as np
from flowField import *
from flowFieldWavy import *

# eps = 10^(-3.+0.025*fileind)   => fileind = 40*(log10(eps) +3)
# g0 = [0.2:0.025:0.6]    => gind = 40*(g-0.2)
# g1 = [0.65:0.05:1.2, 1.3,1.4,1.5]   => gind = 20*(g-0.65)
eps = 0.03/4.;  g = 3.0/2.; Re = 1000
gFileInd = 0
gInd = int(40*(g-0.2))
if g> 0.6:
    gFileInd = 1
    gInd = int(20*(g-0.65))
    if gInd > 11: gInd = int((gInd-11)/2+11)
ReInd = np.int(np.log10(Re)-1)
        
eFileInd = 2*int(40*(np.log10(eps)+3)/2)

folderPath = '/home/sabarish/matData/seprn/'
fileName = 'dataSeprn'+str(gFileInd)+'b90_'+str(eFileInd)+'.mat'

#vfList, pfList,paramDict = data2ff(fName=folderPath+fileName,ind=gInd)
vf, pf,paramDict = data2ff(fName=folderPath+fileName,ind=3*gInd+ReInd)
eps = paramDict['eps']; g = paramDict['g']; Re = paramDict['Re']
printName = 'tecFiles/wavyB90E'+str(-np.log10(eps))+'G'+str(g)+'Re'+str(int(Re))+'.dat'
vf.printPhysicalPlanar(pField=pf, fName=printName)
#gArr = paramDict['g']
#ReArr = paramDict['Re']
#paramDict['g'][3:6]

## Script for printing .dat files for flow-vis (in Tecplot) using flowFieldArray

In [ ]:
eps = 0.2/2.;  g = 0.6/2.; Re = 100
# printing for 2*eps = 0.0045, 0.02, 0.06, 2*g = 0.625, 1.2, 2.0
epsInd = np.argwhere(vArr[:,0,0].getProperty('eps')>=eps)[0,0]
gInd = np.argwhere(vArr[0,:,0].getProperty('g')>=g)[0,0]
ReInd = np.argwhere(vArr[0,0,:].getProperty('Re')>=Re)[0,0]
print(epsInd,gInd,ReInd)
vArrObj = vArr[epsInd, gInd, ReInd]; vf = vArrObj.field; pf = pArr[epsInd,gInd,ReInd].field
eps = vArrObj.eps; g = vArrObj.g; Re = vArrObj.Re

#eps = paramDict['eps']; g = paramDict['g']; Re = paramDict['Re']
printName = 'finalPlots/wavyE'+str(round(-np.log10(eps),2))+'G'+str(round(g,2))+'Re'+str(int(Re))+'.dat'
                                 
vf.printPhysicalPlanar(pField=pf, fName=printName)
print(2*eps,2*g,Re)

## Validation with OpenFOAM, and checking for symmetry

In [ ]:
import numpy as np
from flowFieldWavy import *
from myUtils import *

filePath =  '/home/sabarish/OpenFOAM/sabarish-2.4.x/run/wavyLam/ep02/a15/finest/'
endTime = '400/'
pointsName=filePath+'constant/polyMesh/points'
eps = 0.02
a = 15.
g= a*eps

x,y,z = readPoints(fName=pointsName, nx = 61,ny=501)
yMax =np.max(y)
yMin = np.min(y)
#h = (yMax-yMin)/2.08
h = 0.1
x = x/h; y =(y-h)/h; z=z/h

import matplotlib.pyplot as plt
%matplotlib inline
#vF,pF,paramDict = mapData2ff(g=g, eps=eps, Re=100, theta=0)
vF,pF,fnorm = runMATLAB(g=g,eps=eps,Re=100.,theta=0,N=50,n=5)
#print(paramDict)
print(vF.N, vF.nx)
u = vF.getScalar(nd=0)
v = vF.getScalar(nd=1)
uData = np.zeros(x.shape); vData = np.zeros(x.shape); uUnInt = np.zeros((x.shape[0],x.shape[1],u.N))
for kx in range(x.shape[0]):
    for kz in range(x.shape[1]):
        yBottom = -1.+2.*eps*np.cos(a*x[kx,0,0])
        yT = y[kx,kz]-yBottom-1.
        uData[kx,kz] = u.getPhysical(xLoc=x[kx,0,0], zLoc=z[0,kz,0], yLoc = yT,withBase=True)
        uUnInt[kx,kz] = u.getPhysical(xLoc=x[kx,0,0], zLoc=z[0,kz,0], withBase=True)
        vData[kx,kz] = v.getPhysical(xLoc=x[kx,0,0], zLoc=z[0,kz,0], yLoc = yT)
        
yUnInt = np.zeros((x.shape[0],u.N))
yCheb = chebdif(u.N,1)[0]
for kx in range(x.shape[0]):
    xLoc = x[kx,0,0]
    yUnInt[kx] = yCheb + 2.*u.flowDict['eps']*np.cos(u.flowDict['alpha']*xLoc)
        
#writeFOAMData(vF,pF)
#'''
uFoam,vFoam,wFoam = readFOAMData(fName=filePath+endTime+'U',nx=60,ny=500)
print('max u in uFoam:',np.max(uFoam))
uFoam = uFoam/0.015
vFoam = vFoam/0.015
"""
plt.figure()
plt.title('Error in U')
cLimits = np.arange(0.,0.008,0.0005)
plt.contourf(x[:,0], y[:,0], np.abs(uData[:,0]-uFoam[:,0]),levels=cLimits)
plt.colorbar()
#plt.clim([-0.008,0.008])
plt.savefig('./figures/uDiffG10.pdf', format='pdf')


plt.figure()
plt.title('Relative difference in U (truncated at 2%)')
uRel = np.abs((uData[:,0]-uFoam[:,0])/uFoam[:,0])
uRel[uRel>0.02] = 0.02; uRel[uRel<-0.02] = -0.02
plt.contourf(x[:,0], y[:,0],uRel , 50)
plt.colorbar()
plt.clim([0.,0.02])
plt.savefig('./figures/uRelDiffG10.pdf', format='pdf')

plt.figure()
plt.title('U from OpenFOAM')
plt.contourf(x[:,0], y[:,0], uFoam[:,0], 50)
plt.colorbar()
plt.clim([0.,0.95])
print('max U velocity from foamData:',np.max(uFoam))
plt.savefig('./figures/uFoamG10.pdf', format='pdf')

plt.figure()
plt.title('U from current spectral solver')
plt.contourf(x[:,0], y[:,0], uData[:,0], 50)
plt.colorbar()
plt.clim([0.,1.0])
plt.savefig('./figures/uMatG10.pdf', format='pdf')

plt.figure()
plt.title('OpenFOAM symmetry')
nx = x.shape[0]
u1 = uFoam[:nx//2,0]; u2 = uFoam[nx//2:,0,::-1]
udif = (u2-u1)/u1
udif[udif>0.05]=0.05; udif[udif<-0.05] = -0.05
plt.contourf(x[:nx//2,0], y[:nx//2,0],udif , 50)
plt.colorbar()

plt.figure()
plt.title('MATLAB solution symmetry- Interpolated')
nx = x.shape[0]
u1 = uData[:nx//2,0]; u2 = uData[nx//2:,0,::-1]
udif = (u2-u1)/u1
udif[udif>0.05]=0.05; udif[udif<-0.05] = -0.05
plt.contourf(x[:nx//2,0], y[:nx//2,0],udif , 50)
plt.colorbar()
"""
plt.figure()
plt.title('Flow symmetry:Relative difference in U (truncated at 1%)')
nx = x.shape[0]
u1 = uUnInt[:nx//2,0]; u2 = uUnInt[nx//2:,0,::-1]
#print('a*xLoc/pi for symmetry: ', a*x[:nx//2,0,0]/np.pi)
udif = np.abs((u2-u1)/u1)
udif[udif>0.01]=0.01
plt.contourf(x[:nx//2,0,:u.N], yUnInt[:nx//2],udif , 50)
plt.colorbar()
plt.savefig('./figures/uSymG06.pdf',format='pdf')

"""
plt.figure()
plt.title('Error in V')
plt.contourf(x[:,0], y[:,0], vData[:,0]-vFoam[:,0], 50)
plt.colorbar()
plt.savefig('vDiffG10.pdf',format='pdf')

plt.figure()
plt.title('V from OpenFOAM')
plt.contourf(x[:,0], y[:,0], vFoam[:,0], 50)
plt.colorbar()
plt.savefig('vFoamG10.pdf',format='pdf')

plt.figure()
plt.title('V from current solver')
plt.contourf(x[:,0], y[:,0], vData[:,0], 50)
plt.colorbar()
plt.savefig('vMatG10.pdf',format='pdf')

plt.figure()
plt.title('Relative difference in V (truncated at 2%)')
vRel = np.abs((vData[:,0]-vFoam[:,0])/vFoam[:,0])
vRel[vRel>0.02] = 0.02
plt.contourf(x[:,0], y[:,0],vRel , 50)
plt.colorbar()
plt.savefig('./figures/vRelDiffG10.pdf', format='pdf')

plt.show()

#'''
"""

##Equivalent wall shift

In [ ]:
hEq = np.sqrt(3./2.*uBulk)
epsEq = 1.-hEq

plt.figure()
plt.title('$\\epsilon_{eq}$ at $Re_{\\tau}$='+str(round(np.sqrt(20),1)))
plt.contourf(2.*gArr[:,:,0], 2.*epsArr[:,:,0], epsEq[:,:,0])
plt.colorbar()
plt.xlabel('Slope');plt.ylabel('Amplitude')
plt.savefig('epsEqRe10.pdf',format='pdf')
plt.show()

plt.figure()
plt.title('$\\epsilon_{eq}$ at $Re_{\\tau}$='+str(round(np.sqrt(200),1)))
plt.contourf(2.*gArr[:,:,0], 2.*epsArr[:,:,0], epsEq[:,:,10])
plt.colorbar()
plt.xlabel('Slope');plt.ylabel('Amplitude')
plt.savefig('epsEqRe100.pdf',format='pdf')
plt.show()

plt.figure()
plt.title('$\\epsilon_{eq}$ at $Re_{\\tau}$='+str(round(np.sqrt(2000),1)))
plt.contourf(2.*gArr[:,:,0], 2.*epsArr[:,:,0], epsEq[:,:,20])
plt.colorbar()
plt.xlabel('Slope');plt.ylabel('Amplitude')
plt.savefig('epsEqRe1000.pdf',format='pdf')
plt.show()
